In [19]:
# 구글 드라이브 입력
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [20]:
import xgboost as xgb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 

#from catboost import Pool,CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

import os
import sys
import platform
import random
import math
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest

In [21]:
train = pd.read_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/test.csv')

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

num_train_x, num_train_y = get_x_y(train)
num_test_x = get_x_y(test)

class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

snp_data = []
for col in snp_col:
    snp_data += list(num_train_x[col].values)

num_train_y = class_le.fit_transform(num_train_y)
snp_le.fit(snp_data)

for col in num_train_x.columns:
    if col in snp_col:
        num_train_x[col] = snp_le.transform(num_train_x[col])
        num_test_x[col] = snp_le.transform(num_test_x[col])

In [22]:
clf = IsolationForest(max_samples=100, contamination = 0.03, random_state=42)
clf.fit(num_train_x.iloc[:,range(4,19)].values)

y_iso_train = clf.predict(num_train_x.iloc[:,range(4,19)].values)
num_train_x_iso = num_train_x.loc[y_iso_train == 1]
num_train_y_iso = num_train_y[y_iso_train == 1]

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [23]:
num_train_x_iso

,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,0,0,0,2,5,1,0,4,2,0,0,5,0,5,1,0,0,0,0
1,0,0,0,2,1,1,2,0,0,1,0,4,0,1,0,4,5,0,0
3,0,0,0,1,0,5,0,4,0,5,5,0,5,1,5,5,5,0,5
4,0,0,0,2,5,5,3,0,3,0,0,0,0,5,0,0,1,0,4
5,0,0,0,2,5,5,2,0,3,0,0,4,0,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0,0,0,2,1,1,0,4,3,1,0,4,0,5,1,4,0,0,0
258,0,0,0,2,5,0,2,0,0,1,4,4,0,1,1,0,1,0,4
259,0,0,0,1,1,5,0,4,0,1,5,4,4,0,5,5,5,2,5
260,0,0,0,1,0,5,0,4,0,5,5,0,4,1,1,4,5,2,5


In [24]:
num_train_y_only_A = (num_train_y_iso == 0).astype(int)
num_train_y_only_B = (num_train_y_iso == 1).astype(int)
num_train_y_only_C = (num_train_y_iso == 2).astype(int)

In [25]:
def workit_LGBMClassifier(train_x_using_iso, the_real_train_x, the_real_train_y, what_is_train_y, what_is_test_x,  how_many_sss, nn_ssplit, nnn_estimator, max_ddepth, l_r, w_childsample = 20):
    hundred_df_train = pd.DataFrame({'zeros':np.zeros(len(the_real_train_x))})
    hundred_df_test = pd.DataFrame({'zeros':np.zeros(len(what_is_test_x))})

    for i in range(0,(how_many_sss*nn_ssplit)):
        globals()['s{}'.format(i)] = np.zeros(len(what_is_test_x))
        ya_qustn = 's'+str(i)
        hundred_df_train = pd.concat([hundred_df_train, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)
        hundred_df_test = pd.concat([hundred_df_test, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)

    antms_val = 0
    antms_f1 = 0
    for sss in range(how_many_sss):
        rrr = 0
        seed_everything(sss) # Seed 고정

        cv = KFold(n_splits=nn_ssplit, shuffle=True, random_state=sss)
        mean_val_f1 = 0

        for tri, vai in cv.split(what_is_train_y):
            print(sss, (1 + rrr%nn_ssplit),'번째 시작!')

            input_train = train_x_using_iso.iloc[tri]
            input_val = train_x_using_iso.iloc[vai]

            true_train = what_is_train_y[tri]
            true_val = what_is_train_y[vai]

            globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))] = LGBMClassifier(n_estimators=nnn_estimator, max_depth =max_ddepth, learning_rate =l_r,random_state = sss, min_child_samples = w_childsample) 
            globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].fit(input_train, true_train , early_stopping_rounds=500,  eval_set = [(input_val, true_val)] , eval_metric='logloss', verbose=False)

            output_train = globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_train)
            output_val = globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_val)

            f1_train = f1_score(true_train, output_train, average='macro')
            f1_val = f1_score(true_val, output_val, average='macro')

            # 모델 예측
            mean_val_f1 += f1_val
            print('F1 score of train and val', f1_train, f1_val)

            kkk_train = list(globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].predict(the_real_train_x))
            kkk_test = list(globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].predict(what_is_test_x))

            hundred_df_train.iloc[:,(rrr+1)] = kkk_train
            hundred_df_test.iloc[:,(rrr+1)] = kkk_test
            
            rrr += 1
        print(' ')

        mean_f1_score = mean_val_f1/nn_ssplit
        print('이상치 제거한 것의 평균 f1 score: ', mean_f1_score)
        print('온전한 Train data의 Confusion matrix')
        print(confusion_matrix(the_real_train_y, kkk_train))
        
        print(' ')
    return hundred_df_train, hundred_df_test

In [26]:
how_many_sssss = 1
nn_ssplitss = 5
hh_df = workit_LGBMClassifier(num_train_x_iso, num_train_x, num_train_y, num_train_y_iso, num_test_x, how_many_sssss, nn_ssplitss, 120, -1, 0.12)
submission = pd.read_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/sample_submission.csv')
submission.iloc[:,1] = np.round(np.sum(hh_df[1].iloc[:,range(1,(how_many_sssss*nn_ssplitss + 1))],axis=1)/(how_many_sssss*nn_ssplitss))
submission = submission.replace(0,'A')
submission = submission.replace(1,'B')
submission = submission.replace(2,'C')
submission.to_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/dkwn_1.csv', index=False)

0 1 번째 시작!
F1 score of train and val 1.0 0.9801307189542484
0 2 번째 시작!
F1 score of train and val 1.0 0.9257142857142857
0 3 번째 시작!
F1 score of train and val 1.0 0.980238302818948
0 4 번째 시작!
F1 score of train and val 0.9904067284315659 0.949206349206349
0 5 번째 시작!
F1 score of train and val 1.0 1.0
 
이상치 제거한 것의 평균 f1 score:  0.9670579313387663
온전한 Train data의 Confusion matrix
[[ 69   0   0]
 [  0 114   0]
 [  0   0  79]]
 


In [27]:
how_many_sssss = 1
nn_ssplitss = 5
after_train_A, after_test_A = workit_LGBMClassifier(num_train_x_iso, num_train_x, (num_train_y == 0).astype(int), num_train_y_only_A, num_test_x, how_many_sssss, nn_ssplitss, 50, 3, 0.12)
after_train_A = np.array(after_train_A.iloc[:,range(1,6)].mean(axis=1))
after_test_A = np.array(after_test_A.iloc[:,range(1,6)].mean(axis=1))

0 1 번째 시작!
F1 score of train and val 1.0 1.0
0 2 번째 시작!
F1 score of train and val 1.0 1.0
0 3 번째 시작!
F1 score of train and val 1.0 1.0
0 4 번째 시작!
F1 score of train and val 1.0 1.0
0 5 번째 시작!
F1 score of train and val 1.0 1.0
 
이상치 제거한 것의 평균 f1 score:  1.0
온전한 Train data의 Confusion matrix
[[193   0]
 [  0  69]]
 


In [28]:
except_a_train_x = num_train_x.iloc[after_train_A == 0]
except_a_train_y = num_train_y[after_train_A == 0]
except_a_test_x = num_test_x[after_test_A == 0]
except_a_train_y_getting_c = (except_a_train_y == 2).astype(int)

In [29]:
from sklearn.preprocessing import MinMaxScaler
snp = pd.read_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/snp_info.csv')

min_max_scaler = MinMaxScaler()
snp_scaled = min_max_scaler.fit_transform(snp.iloc[:,[2,3,4]])
snp_scaled

array([[0.        , 0.684244  , 0.46004101],
       [0.5       , 0.30789798, 0.14271385],
       [0.5       , 0.69718728, 0.47817817],
       [0.5       , 0.79768174, 0.57828045],
       [0.5       , 0.82521846, 0.66504086],
       [0.5       , 0.82620289, 0.67888345],
       [0.5       , 0.8281382 , 0.68967038],
       [0.5       , 0.84811611, 0.72755847],
       [0.5       , 0.89202641, 0.78845199],
       [0.625     , 0.63197811, 0.43648101],
       [0.75      , 1.        , 1.        ],
       [0.875     , 0.63907669, 0.59117187],
       [0.875     , 0.6461197 , 0.64227843],
       [0.875     , 0.68178136, 0.78550369],
       [1.        , 0.        , 0.        ]])

In [30]:
clf2 = IsolationForest(max_samples=100, contamination = 0.03, random_state=42)
clf2.fit(except_a_train_x.iloc[:,range(4,19)].values)

except_a_iso_train_y_value = clf2.predict(except_a_train_x.iloc[:,range(4,19)].values)
except_a_train_x_iso = except_a_train_x.loc[except_a_iso_train_y_value == 1]
except_a_train_y_getting_c_iso = except_a_train_y_getting_c[except_a_iso_train_y_value == 1]

In [31]:
whole_train_ele_iso = []
whole_test_ele = []

for jj in range(len(except_a_train_x_iso)):
    each_train_ele = []
    for kk in range(15):
        each_snp_chrom = list(snp_scaled[kk]) + [except_a_train_x_iso.iloc[jj,(4+kk)],except_a_train_x_iso.iloc[jj,3]]
        each_train_ele.append(each_snp_chrom)
    whole_train_ele_iso.append(each_train_ele)
whole_train_ele_iso = torch.tensor(whole_train_ele_iso)
print(whole_train_ele_iso.shape)

for jj in range(len(except_a_test_x)):
    each_test_ele = []
    for kk in range(15):
        each_snp_chrom = list(snp_scaled[kk]) + [except_a_test_x.iloc[jj,(4+kk)],except_a_test_x.iloc[jj,3]]
        each_test_ele.append(each_snp_chrom)
    whole_test_ele.append(each_test_ele)
whole_test_ele = torch.tensor(whole_test_ele)

print(whole_test_ele.shape)

train_y_df_iso = pd.DataFrame({'B':np.zeros(len(except_a_train_y_getting_c_iso)), 'C':np.zeros(len(except_a_train_y_getting_c_iso))})

for kk in range(len(except_a_train_y_getting_c_iso)):
    if except_a_train_y_getting_c_iso[kk] == 0:
        train_y_df_iso.iloc[kk,0] = 1
    elif except_a_train_y_getting_c_iso[kk] == 1:
        train_y_df_iso.iloc[kk,1] = 1

print(train_y_df_iso)


torch.Size([187, 15, 5])
torch.Size([124, 15, 5])
       B    C
0    1.0  0.0
1    0.0  1.0
2    0.0  1.0
3    1.0  0.0
4    1.0  0.0
..   ...  ...
182  1.0  0.0
183  1.0  0.0
184  1.0  0.0
185  0.0  1.0
186  1.0  0.0

[187 rows x 2 columns]


In [ ]:
train_y_df_iso.sum(axis=0)

B    111.0
C     76.0
dtype: float64

In [32]:
import copy
from sklearn.svm import NuSVC

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=15, out_channels=256, kernel_size = 3)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size = 2)
        self.fc1 = nn.Linear(1024, 256)
        self.bn1d1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.bn1d2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 16)
        self.bn1d3 = nn.BatchNorm1d(16)
        self.fc4 = nn.Linear(16,2)
        self.leakyrelu = nn.LeakyReLU()

    def forward(self, x2):
        x2 = self.conv1(x2)
        x2 = self.conv2(x2)
        x2 = x2.view(len(x2),-1)
        x = self.fc1(x2)
        x = self.bn1d1(x)
        x = self.leakyrelu(x)
        x = self.fc2(x)
        x = self.bn1d2(x)
        x = self.leakyrelu(x)
        x = self.fc3(x)
        x = self.bn1d3(x)
        x = self.leakyrelu(x)
        x = self.fc4(x)

        return x, x2

net = Net()

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = net.parameters(), lr=0.01)

hundred_df = pd.DataFrame({'임시': np.zeros(len(whole_test_ele))})

how_many_sss = 1
nn_ssplit = 5
for i in range(0,(how_many_sss*nn_ssplit)):
    globals()['s{}'.format(i)] = np.zeros(len(whole_test_ele))
    ya_qustn = 's'+str(i)
    hundred_df = pd.concat([hundred_df, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)

rrr = 0
antms_val = 0
antms_f1 = 0
for sss in range(how_many_sss):
    seed_everything(sss) # Seed 고정
    criterion = nn.CrossEntropyLoss()
    net = Net()
    optimizer = optim.AdamW(params = net.parameters(), lr=0.01, amsgrad=True)

    cv = KFold(n_splits=nn_ssplit, shuffle=True, random_state=sss)
    mean_val_f1 = 0

    for tri, vai in cv.split(train_y_df_iso):
        print(sss, (1 + rrr%nn_ssplit),'번째 시작!')
        earlystop = 0
        val_loss = 100000000000
        f1_loss = 0
        for j in range(50000):    
            input_train = whole_train_ele_iso.type(torch.float32)[[tri]]
            output_train, experiment_train = net(input_train)
            true_tensor_train = torch.tensor(except_a_train_y_getting_c_iso)[tri]
            loss_train = criterion(output_train, true_tensor_train)

            clf_train = NuSVC(nu=0.42, kernel='rbf', gamma ='scale' ,random_state = sss)
            clf_train.fit(pd.DataFrame(output_train.detach().numpy()), except_a_train_y_getting_c_iso[tri])

            with torch.no_grad():
                input_val = whole_train_ele_iso.type(torch.float32)[[vai]]
                output_val, experiment_val = net(input_val)
                true_tensor_val = torch.tensor(except_a_train_y_getting_c_iso)[vai]
                loss_var = criterion(output_val, true_tensor_val)
                f1_var = (f1_score(list(np.array(except_a_train_y_getting_c_iso)[vai]), list(clf_train.predict(pd.DataFrame(output_val.detach().numpy()))), average='macro'))

                if j <= 9:
                    earlystop = 0
                else:
                    if float(val_loss) >= float(loss_var):
                        val_loss = float(loss_var)
                        earlystop = 0
                        antms_val = j
                        if float(f1_loss) < float(f1_var):
                            f1_loss = float(f1_var)
                            antms_f1 = j    
                            new_net = copy.deepcopy(net)
                            clf_val = copy.deepcopy(clf_train)  
                    else:
                        if float(f1_loss) <= float(f1_var):
                            f1_loss = f1_var                    
                            earlystop = 0
                            antms_f1 = j
                            new_net = copy.deepcopy(net)
                            clf_val = copy.deepcopy(clf_train)  
                        else:
                            earlystop += 1

            if earlystop >= 501:
                print(j,'에서 멈춤')
                print('loss_train: ',loss_train)
                print('loss_val: ',loss_var)
                print('최소 loss_val: ', val_loss, antms_val)
                print('최대 f1_loss: ', f1_loss, antms_f1)
                break

            optimizer.zero_grad()  
            loss_train.backward()
            optimizer.step()

        with torch.no_grad():
            output_train_for_xg, experiment_train_for_xg = new_net(whole_train_ele_iso.type(torch.float32)[[tri]])
            experiment_train_for_xg = pd.DataFrame(experiment_train_for_xg.detach().numpy())
            output_train_for_xg = pd.DataFrame(output_train_for_xg.detach().numpy())

            output_val_for_xg, experiment_val_for_xg = new_net(whole_train_ele_iso.type(torch.float32)[[vai]])
            experiment_val_for_xg = pd.DataFrame(experiment_val_for_xg.detach().numpy())
            output_val_for_xg = pd.DataFrame(output_val_for_xg.detach().numpy())

            output_test, experiment_test = new_net(whole_test_ele.type(torch.float32))
            experiment_test = pd.DataFrame(experiment_test.detach().numpy())
            output_test = pd.DataFrame(output_test.detach().numpy())


        # 모델 예측
        the_one_val_f1 = (f1_score(list(np.array(except_a_train_y_getting_c_iso)[vai]), list(clf_val.predict(output_val_for_xg)), average='macro'))
        mean_val_f1 += the_one_val_f1
        print(the_one_val_f1)

        kkk = list(clf_val.predict(output_test))
        hundred_df.iloc[:,(rrr+1)] = kkk
        rrr += 1
        print(' ')

    mean_f1_score = mean_val_f1/nn_ssplit
    print(' ')
    print(mean_f1_score)

    print(' ')

0 1 번째 시작!
4092 에서 멈춤
loss_train:  tensor(3.4704e-05, grad_fn=<NllLossBackward0>)
loss_val:  tensor(0.0249)
최소 loss_val:  0.004314076155424118 52
최대 f1_loss:  1.0 3591
1.0
 
0 2 번째 시작!
16099 에서 멈춤
loss_train:  tensor(0.0002, grad_fn=<NllLossBackward0>)
loss_val:  tensor(0.5763)
최소 loss_val:  0.1485133022069931 15349
최대 f1_loss:  0.9732205778717407 15598
0.9732205778717407
 
0 3 번째 시작!
22846 에서 멈춤
loss_train:  tensor(9.6619e-05, grad_fn=<NllLossBackward0>)
loss_val:  tensor(0.2661)
최소 loss_val:  0.14941339194774628 11
최대 f1_loss:  0.9716475095785441 22345
0.9716475095785441
 
0 4 번째 시작!
3433 에서 멈춤
loss_train:  tensor(8.8385e-05, grad_fn=<NllLossBackward0>)
loss_val:  tensor(0.6075)
최소 loss_val:  0.28044503927230835 20
최대 f1_loss:  0.9439393939393939 2932
0.9439393939393939
 
0 5 번째 시작!
511 에서 멈춤
loss_train:  tensor(7.8526e-05, grad_fn=<NllLossBackward0>)
loss_val:  tensor(0.5535)
최소 loss_val:  0.2114798128604889 10
최대 f1_loss:  0.9407051282051282 10
0.9407051282051282
 
 
0.965902521918

In [34]:
submission_dnn_NuSVC_1 = pd.read_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/sample_submission.csv')
submission_dnn_NuSVC_1.iloc[after_test_A == 0, 1] = round(hundred_df.iloc[:,range(1,6)].mean(axis=1))
submission_dnn_NuSVC_1 = submission_dnn_NuSVC_1.replace(0,'B')
submission_dnn_NuSVC_1 = submission_dnn_NuSVC_1.replace(1,'C')
print(submission_dnn_NuSVC_1)
submission_dnn_NuSVC_1.to_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/submission_dnn_NuSVC_1.csv', index=False)

           id class
0    TEST_000     A
1    TEST_001     B
2    TEST_002     C
3    TEST_003     C
4    TEST_004     A
..        ...   ...
170  TEST_170     B
171  TEST_171     C
172  TEST_172     C
173  TEST_173     B
174  TEST_174     B

[175 rows x 2 columns]


In [ ]:
workit_LGBMClassifier(num_train_x_iso, num_train_x, (num_train_y == 0).astype(int), num_train_y_only_A, num_test_x, how_many_sssss, nn_ssplitss, 50, 3, 0.12)

In [ ]:
def workit_RandomForestClassifier(train_x_using_iso, the_real_train_x, the_real_train_y, what_is_train_y, what_is_test_x,  how_many_sss, nn_ssplit, nnn_estimator, max_ddepth):
    hundred_df_train = pd.DataFrame({'zeros':np.zeros(len(the_real_train_x))})
    hundred_df_test = pd.DataFrame({'zeros':np.zeros(len(what_is_test_x))})

    for i in range(0,(how_many_sss*nn_ssplit)):
        globals()['s{}'.format(i)] = np.zeros(len(what_is_test_x))
        ya_qustn = 's'+str(i)
        hundred_df_train = pd.concat([hundred_df_train, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)
        hundred_df_test = pd.concat([hundred_df_test, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)

    antms_val = 0
    antms_f1 = 0
    for sss in range(how_many_sss):
        rrr = 0
        seed_everything(sss) # Seed 고정

        cv = KFold(n_splits=nn_ssplit, shuffle=True, random_state=sss)
        mean_val_f1 = 0

        for tri, vai in cv.split(what_is_train_y):
            print(sss, (1 + rrr%nn_ssplit),'번째 시작!')

            globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))] = RandomForestClassifier(n_estimators=nnn_estimator, max_depth =max_ddepth,random_state = sss) 
            input_train = train_x_using_iso.iloc[tri]
            input_val = train_x_using_iso.iloc[vai]

            true_train = what_is_train_y[tri]
            true_val = what_is_train_y[vai]

            globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].fit(input_train, true_train)
            output_train = globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_train)
            output_val = globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_val)

            f1_train_iso = f1_score(true_train, output_train, average='macro')
            f1_val_iso = f1_score(true_val, output_val, average='macro')

            # 모델 예측
            mean_val_f1 += f1_val_iso
            print('F1 score of train and val', f1_train_iso, f1_val_iso)

            kkk_train = list(globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].predict(the_real_train_x))
            kkk_test = list(globals()['lgbm_train_{}'.format((1 + rrr%nn_ssplit))].predict(what_is_test_x))

            hundred_df_train.iloc[:,(rrr+1)] = kkk_train
            hundred_df_test.iloc[:,(rrr+1)] = kkk_test
            
            rrr += 1
        print(' ')

        mean_f1_score = mean_val_f1/nn_ssplit
        print('이상치 제거한 것의 평균 f1 score: ', mean_f1_score)
        print('온전한 Train data의 Confusion matrix')
        print(confusion_matrix(the_real_train_y, kkk_train))
        
        print(' ')
    return hundred_df_train, hundred_df_test

In [ ]:
except_a_train_x

,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,0,0,0,2,5,1,0,4,2,0,0,5,0,5,1,0,0,0,0
1,0,0,0,2,1,1,2,0,0,1,0,4,0,1,0,4,5,0,0
2,0,0,0,2,5,5,0,4,3,5,0,4,4,1,0,0,0,0,0
4,0,0,0,2,5,5,3,0,3,0,0,0,0,5,0,0,1,0,4
5,0,0,0,2,5,5,2,0,3,0,0,4,0,5,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,0,0,0,2,1,5,2,4,3,1,4,5,0,5,5,0,0,0,4
256,0,0,0,2,5,5,2,4,3,1,0,5,0,5,1,4,0,0,0
257,0,0,0,2,1,1,0,4,3,1,0,4,0,5,1,4,0,0,0
258,0,0,0,2,5,0,2,0,0,1,4,4,0,1,1,0,1,0,4


In [ ]:
clf_checking_b = IsolationForest(max_samples=50, contamination = 0.05, random_state=42)
clf_checking_c = IsolationForest(max_samples=50, contamination = 0.05, random_state=42)
except_a_train_x_only_b = except_a_train_x.iloc[except_a_train_y_getting_c == 0,range(4,19)]
except_a_train_x_only_c = except_a_train_x.iloc[except_a_train_y_getting_c == 1,range(4,19)]
who_is_outier_b = clf_checking_b.fit_predict(except_a_train_x_only_b.values)
who_is_outier_c = clf_checking_c.fit_predict(except_a_train_x_only_c.values)
except_a_train_x.iloc[:,0] = except_a_train_y_getting_c
bc_iso_train_x = except_a_train_x.drop(index = except_a_train_x_only_b.iloc[who_is_outier_b == (-1)].index) 
bc_iso_train_x = bc_iso_train_x.drop(index = except_a_train_x_only_c.iloc[who_is_outier_c == (-1)].index)
bc_iso_train_y = np.array(bc_iso_train_x.iloc[:,0])
bc_iso_train_x.iloc[:,0] = np.zeros(len(bc_iso_train_x))
bc_iso_train_x = bc_iso_train_x.astype(int)
except_a_train_x.iloc[:,0]  = np.zeros(len(except_a_train_x))
except_a_train_x = except_a_train_x.astype(int)

doing_onehot = pd.concat([bc_iso_train_x , except_a_train_x], axis=0)
doing_onehot = pd.concat([doing_onehot , except_a_test_x], axis=0)

doing_onehot = pd.concat([bc_iso_train_x , except_a_train_x], axis=0)
doing_onehot = pd.concat([doing_onehot , except_a_test_x], axis=0)
doing_onehot= pd.get_dummies(doing_onehot.astype(str))
onehot_bc_iso_train_x = doing_onehot.iloc[range(0,len(bc_iso_train_x))]
onehot_except_a_train_x = doing_onehot.iloc[range(len(bc_iso_train_x), (len(bc_iso_train_x) + len(except_a_train_x)))]
onehot_except_a_test_x = doing_onehot.iloc[range(len(bc_iso_train_x) + len(except_a_train_x), (len(bc_iso_train_x)  + len(except_a_train_x) + len(except_a_test_x)))]

#workit_LGBMClassifier(train_x_using_iso, the_real_train_x, the_real_train_y, what_is_train_y, what_is_test_x,  how_many_sss, nn_ssplit, nnn_estimator, max_ddepth, l_r, w_childsample = 20)
workit_LGBMClassifier(onehot_bc_iso_train_x, onehot_except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, onehot_except_a_test_x, how_many_sssss, nn_ssplitss, 10000, 2, 0.15, 2)

0 1 번째 시작!
F1 score of train and val 1.0 0.9645933014354067
0 2 번째 시작!
F1 score of train and val 1.0 0.972972972972973
0 3 번째 시작!
F1 score of train and val 1.0 0.917960088691796
0 4 번째 시작!
F1 score of train and val 0.9929016369694336 0.916083916083916
0 5 번째 시작!
F1 score of train and val 1.0 1.0
 
이상치 제거한 것의 평균 f1 score:  0.9543220558368184
온전한 Train data의 Confusion matrix
[[114   0]
 [  1  78]]
 


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   1   0   1   1   1
 4      0.0   0   0   0   0   0
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   0
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   0   0   0   0
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   0
 120    0.0   1   1   1   1   1
 121    0.0   1   1   1   1   1
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
bc_iso_train_x[bc_iso_train_x.index == 0].iloc[:,4][0] == 5

True

{'SNP_09_5'}

In [ ]:
submission_twice_lgbm = pd.read_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/sample_submission.csv')
submission_twice_lgbm.iloc[after_test_A == 0, 1] = round(workit_LGBMClassifier(onehot_bc_iso_train_x, onehot_except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, onehot_except_a_test_x, how_many_sssss, nn_ssplitss, 10000, 2, 0.15, 2)[1].iloc[:,range(1,6)].mean(axis=1))
submission_twice_lgbm = submission_twice_lgbm.replace(0,'B')
submission_twice_lgbm = submission_twice_lgbm.replace(1,'C')
submission_twice_lgbm.to_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/submission_twice_lgbm_2.csv', index=False)

0 1 번째 시작!
F1 score of train and val 1.0 0.9645933014354067
0 2 번째 시작!
F1 score of train and val 1.0 0.972972972972973
0 3 번째 시작!
F1 score of train and val 1.0 0.917960088691796
0 4 번째 시작!
F1 score of train and val 0.9929016369694336 0.916083916083916
0 5 번째 시작!
F1 score of train and val 1.0 1.0
 
이상치 제거한 것의 평균 f1 score:  0.9543220558368184
온전한 Train data의 Confusion matrix
[[114   0]
 [  1  78]]
 


In [ ]:
workit_RandomForestClassifier(bc_iso_train_x, except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, except_a_test_x, how_many_sssss, nn_ssplitss, 200, 4)

0 1 번째 시작!
F1 score of train and val 0.965207100591716 0.969795918367347
0 2 번째 시작!
F1 score of train and val 0.9789327855539101 0.9425465838509317
0 3 번째 시작!
F1 score of train and val 0.9788235294117646 0.9093877551020408
0 4 번째 시작!
F1 score of train and val 0.9709486166007906 0.972972972972973
0 5 번째 시작!
F1 score of train and val 0.9786692289214508 0.9150275373721479
 
이상치 제거한 것의 평균 f1 score:  0.941946153533088
온전한 Train data의 Confusion matrix
[[113   1]
 [  6  73]]
 


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   0   0   0   0   0
 4      0.0   0   0   0   0   0
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   0
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   0   0   0   0
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   0
 120    0.0   1   1   1   1   1
 121    0.0   1   1   1   1   1
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
clf_checking_b = IsolationForest(max_samples=50, contamination = 0.005, random_state=42)
clf_checking_c = IsolationForest(max_samples=50, contamination = 0.045, random_state=42)
except_a_train_x_only_b = except_a_train_x.iloc[except_a_train_y_getting_c == 0,range(4,19)]
except_a_train_x_only_c = except_a_train_x.iloc[except_a_train_y_getting_c == 1,range(4,19)]
who_is_outier_b = clf_checking_b.fit_predict(except_a_train_x_only_b.values)
who_is_outier_c = clf_checking_c.fit_predict(except_a_train_x_only_c.values)
except_a_train_x.iloc[:,0] = except_a_train_y_getting_c
bc_iso_train_x = except_a_train_x.drop(index = except_a_train_x_only_b.iloc[who_is_outier_b == (-1)].index) 
bc_iso_train_x = bc_iso_train_x.drop(index = except_a_train_x_only_c.iloc[who_is_outier_c == (-1)].index)
bc_iso_train_y = np.array(bc_iso_train_x.iloc[:,0])
bc_iso_train_x.iloc[:,0] = np.zeros(len(bc_iso_train_x))
bc_iso_train_x = bc_iso_train_x.astype(int)
except_a_train_x.iloc[:,0]  = np.zeros(len(except_a_train_x))
except_a_train_x = except_a_train_x.astype(int)

doing_onehot = pd.concat([bc_iso_train_x , except_a_train_x], axis=0)
doing_onehot = pd.concat([doing_onehot , except_a_test_x], axis=0)

doing_onehot = pd.concat([bc_iso_train_x , except_a_train_x], axis=0)
doing_onehot = pd.concat([doing_onehot , except_a_test_x], axis=0)
doing_onehot= pd.get_dummies(doing_onehot.astype(str))
onehot_bc_iso_train_x = doing_onehot.iloc[range(0,len(bc_iso_train_x))]
onehot_except_a_train_x = doing_onehot.iloc[range(len(bc_iso_train_x), (len(bc_iso_train_x) + len(except_a_train_x)))]
onehot_except_a_test_x = doing_onehot.iloc[range(len(bc_iso_train_x) + len(except_a_train_x), (len(bc_iso_train_x)  + len(except_a_train_x) + len(except_a_test_x)))]

workit_RandomForestClassifier(onehot_bc_iso_train_x, onehot_except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, onehot_except_a_test_x, how_many_sssss, nn_ssplitss, 2000, 4)

0 1 번째 시작!
F1 score of train and val 0.9649712764466862 0.9721203228173148
0 2 번째 시작!
F1 score of train and val 0.9649712764466862 0.9434523809523809
0 3 번째 시작!
F1 score of train and val 0.9642499642499642 0.9732205778717407
0 4 번째 시작!
F1 score of train and val 0.9585164835164834 0.9407051282051282
0 5 번째 시작!
F1 score of train and val 0.9790713790713791 0.8582375478927204
 
이상치 제거한 것의 평균 f1 score:  0.937547191547857
온전한 Train data의 Confusion matrix
[[112   2]
 [  7  72]]
 


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   0   0   0   0   0
 4      0.0   0   0   0   0   0
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   0
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   0   0   0   0
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   0
 120    0.0   1   1   1   1   1
 121    0.0   1   1   1   1   1
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
submission_lgbm_rf = pd.read_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/sample_submission.csv')
submission_lgbm_rf.iloc[after_test_A == 0, 1] = round(workit_RandomForestClassifier(except_a_train_x_iso,except_a_train_x, except_a_train_y_getting_c_iso, except_a_test_x, how_many_sssss, nn_ssplitss, 200, 4)[1].iloc[:,range(1,6)].mean(axis=1))
submission_lgbm_rf = submission_lgbm_rf.replace(0,'B')
submission_lgbm_rf = submission_lgbm_rf.replace(1,'C')
submission_lgbm_rf.to_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/submission_lgbm_rf_1.csv', index=False)

0 1 번째 시작!
F1 score of train and val 0.965025116191728 0.9727598566308244
0 2 번째 시작!
F1 score of train and val 0.9715865751334859 1.0
0 3 번째 시작!
F1 score of train and val 0.9791076651655137 0.9425465838509317
0 4 번째 시작!
F1 score of train and val 0.9722222222222222 0.8878787878787877
0 5 번째 시작!
F1 score of train and val 0.9655505029626568 0.8814102564102564
 
 
0.9369190969541601
 


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
def workit_KNeighborsClassifier(train_x_using_iso, the_real_train_x, the_real_train_y, what_is_train_y, what_is_test_x,  how_many_sss, nn_ssplit, nnn_neighbor, weight_ele='uniform', the_metirc='minkowski', the_leaf = 30):
    hundred_df_train = pd.DataFrame({'zeros':np.zeros(len(the_real_train_x))})
    hundred_df_test = pd.DataFrame({'zeros':np.zeros(len(what_is_test_x))})

    for i in range(0,(how_many_sss*nn_ssplit)):
        globals()['s{}'.format(i)] = np.zeros(len(what_is_test_x))
        ya_qustn = 's'+str(i)
        hundred_df_train = pd.concat([hundred_df_train, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)
        hundred_df_test = pd.concat([hundred_df_test, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)

    antms_val = 0
    antms_f1 = 0
    for sss in range(how_many_sss):
        rrr = 0
        seed_everything(sss) # Seed 고정

        cv = KFold(n_splits=nn_ssplit, shuffle=True, random_state=sss)
        mean_val_f1 = 0

        for tri, vai in cv.split(what_is_train_y):
            print(sss, (1 + rrr%nn_ssplit),'번째 시작!')

            globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))] = KNeighborsClassifier(n_neighbors=nnn_neighbor, weights =weight_ele, metric =the_metirc, leaf_size = the_leaf)  #,random_state = sss
            input_train = train_x_using_iso.iloc[tri]
            input_val = train_x_using_iso.iloc[vai]

            true_train = what_is_train_y[tri]
            true_val = what_is_train_y[vai]

            globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].fit(input_train, true_train)
            output_train = globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_train)
            output_val = globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_val)

            f1_train = f1_score(true_train, output_train, average='macro')
            f1_val = f1_score(true_val, output_val, average='macro')

            # 모델 예측
            mean_val_f1 += f1_val
            print('F1 score of train and val', f1_train, f1_val)

            kkk_train = list(globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(the_real_train_x))
            kkk_test = list(globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(what_is_test_x))

            hundred_df_train.iloc[:,(rrr+1)] = kkk_train
            hundred_df_test.iloc[:,(rrr+1)] = kkk_test
            
            rrr += 1
        print(' ')

        mean_f1_score = mean_val_f1/nn_ssplit
        print('이상치 제거한 것의 평균 f1 score: ', mean_f1_score)
        print('온전한 Train data의 Confusion matrix')
        print(confusion_matrix(the_real_train_y, kkk_train))
        
        print(' ')
    return hundred_df_train, hundred_df_test

In [ ]:
clf_checking_b = IsolationForest(max_samples=50, contamination = 0.06, random_state=42)
clf_checking_c = IsolationForest(max_samples=50, contamination = 0.02, random_state=42)
except_a_train_x_only_b = except_a_train_x.iloc[except_a_train_y_getting_c == 0,range(4,19)]
except_a_train_x_only_c = except_a_train_x.iloc[except_a_train_y_getting_c == 1,range(4,19)]
who_is_outier_b = clf_checking_b.fit_predict(except_a_train_x_only_b.values)
who_is_outier_c = clf_checking_c.fit_predict(except_a_train_x_only_c.values)
except_a_train_x.iloc[:,0] = except_a_train_y_getting_c
bc_iso_train_x = except_a_train_x.drop(index = except_a_train_x_only_b.iloc[who_is_outier_b == (-1)].index) 
bc_iso_train_x = bc_iso_train_x.drop(index = except_a_train_x_only_c.iloc[who_is_outier_c == (-1)].index)
bc_iso_train_y = np.array(bc_iso_train_x.iloc[:,0])
bc_iso_train_x.iloc[:,0] = np.zeros(len(bc_iso_train_x))

workit_KNeighborsClassifier(bc_iso_train_x, except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, except_a_test_x, how_many_sssss, nn_ssplitss, 3, 'distance', 'euclidean')
# , 'distance', 'euclidean'

0 1 번째 시작!
F1 score of train and val 1.0 0.9002695417789758
0 2 번째 시작!
F1 score of train and val 1.0 0.9149425287356321
0 3 번째 시작!
F1 score of train and val 1.0 0.8766666666666667
0 4 번째 시작!
F1 score of train and val 1.0 0.8377192982456141
0 5 번째 시작!
F1 score of train and val 1.0 0.8583792289535799
 
이상치 제거한 것의 평균 f1 score:  0.8775954528760936
온전한 Train data의 Confusion matrix
[[112   2]
 [  6  73]]
 


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   1   1   1   1   1
 4      0.0   0   0   0   0   0
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   0
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   0   0   0   0
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   1   1   1   1
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   0
 120    0.0   1   1   1   1   1
 121    0.0   0   0   0   1   0
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
from sklearn.linear_model import SGDClassifier
def workit_SGDClassifier(train_x_using_iso, the_real_train_x, the_real_train_y, what_is_train_y, what_is_test_x,  how_many_sss, nn_ssplit, the_loss = 'hinge', the_penalty='l2', the_alpha=0.0001, the_max_iter = 1000):
    hundred_df_train = pd.DataFrame({'zeros':np.zeros(len(the_real_train_x))})
    hundred_df_test = pd.DataFrame({'zeros':np.zeros(len(what_is_test_x))})

    for i in range(0,(how_many_sss*nn_ssplit)):
        globals()['s{}'.format(i)] = np.zeros(len(what_is_test_x))
        ya_qustn = 's'+str(i)
        hundred_df_train = pd.concat([hundred_df_train, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)
        hundred_df_test = pd.concat([hundred_df_test, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)

    antms_val = 0
    antms_f1 = 0
    for sss in range(how_many_sss):
        rrr = 0
        seed_everything(sss) # Seed 고정

        cv = KFold(n_splits=nn_ssplit, shuffle=True, random_state=sss)
        mean_val_f1 = 0

        for tri, vai in cv.split(what_is_train_y):
            print(sss, (1 + rrr%nn_ssplit),'번째 시작!')

            globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))] = SGDClassifier(loss=the_loss, penalty =the_penalty, alpha =the_alpha, max_iter = the_max_iter,random_state = sss)
            input_train = train_x_using_iso.iloc[tri]
            input_val = train_x_using_iso.iloc[vai]

            true_train = what_is_train_y[tri]
            true_val = what_is_train_y[vai]

            globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].fit(input_train, true_train)
            output_train = globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_train)
            output_val = globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_val)

            f1_train = f1_score(true_train, output_train, average='macro')
            f1_val = f1_score(true_val, output_val, average='macro')

            # 모델 예측
            mean_val_f1 += f1_val
            print('F1 score of train and val', f1_train, f1_val)

            kkk_train = list(globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(the_real_train_x))
            kkk_test = list(globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(what_is_test_x))

            hundred_df_train.iloc[:,(rrr+1)] = kkk_train
            hundred_df_test.iloc[:,(rrr+1)] = kkk_test
            
            rrr += 1
        print(' ')

        mean_f1_score = mean_val_f1/nn_ssplit
        print('이상치 제거한 것의 평균 f1 score: ', mean_f1_score)
        print('온전한 Train data의 Confusion matrix')
        print(confusion_matrix(the_real_train_y, kkk_train))
        
        print(' ')
    return hundred_df_train, hundred_df_test

In [ ]:
clf2 = IsolationForest(max_samples=100, contamination = 0.0001, random_state=42)
clf2.fit(except_a_train_x.iloc[:,range(4,19)].values)

except_a_iso_train_y_value = clf2.predict(except_a_train_x.iloc[:,range(4,19)].values)
except_a_train_x_iso = except_a_train_x.loc[except_a_iso_train_y_value == 1]
except_a_train_y_getting_c_iso = except_a_train_y_getting_c[except_a_iso_train_y_value == 1]

workit_SGDClassifier(bc_iso_train_x, except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, except_a_test_x, how_many_sssss, nn_ssplitss, 'hinge','l2',0.0001,100000)

0 1 번째 시작!
F1 score of train and val 0.9792069404498089 0.9353146853146852
0 2 번째 시작!
F1 score of train and val 0.9450056116722783 0.9167291822955739
0 3 번째 시작!
F1 score of train and val 0.9505882352941177 0.9708431836091411
0 4 번째 시작!
F1 score of train and val 0.9855769230769231 0.9726533628972653
0 5 번째 시작!
F1 score of train and val 0.7344375431331953 0.7482517482517483
 
이상치 제거한 것의 평균 f1 score:  0.9087584324736827
온전한 Train data의 Confusion matrix
[[63 51]
 [ 1 78]]
 


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   1   1   0   1   1
 4      0.0   0   0   0   0   1
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   1
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   1   0   0   1
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   1   1   0   1   1
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   1
 120    0.0   1   1   1   1   1
 121    0.0   1   1   1   1   1
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
from sklearn.svm import NuSVC

def workit_NuSVC(train_x_using_iso, the_real_train_x, the_real_train_y, what_is_train_y, what_is_test_x,  how_many_sss, nn_ssplit, the_nu=0.5, the_kernel = 'rbf', the_gamma = 'scale'):
    hundred_df_train = pd.DataFrame({'zeros':np.zeros(len(the_real_train_x))})
    hundred_df_test = pd.DataFrame({'zeros':np.zeros(len(what_is_test_x))})

    for i in range(0,(how_many_sss*nn_ssplit)):
        globals()['s{}'.format(i)] = np.zeros(len(what_is_test_x))
        ya_qustn = 's'+str(i)
        hundred_df_train = pd.concat([hundred_df_train, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)
        hundred_df_test = pd.concat([hundred_df_test, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)

    antms_val = 0
    antms_f1 = 0
    for sss in range(how_many_sss):
        rrr = 0
        seed_everything(sss) # Seed 고정

        cv = KFold(n_splits=nn_ssplit, shuffle=True, random_state=sss)
        mean_val_f1 = 0

        for tri, vai in cv.split(what_is_train_y):
            print(sss, (1 + rrr%nn_ssplit),'번째 시작!')

            globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))] = NuSVC(nu=the_nu, kernel=the_kernel, gamma =the_gamma ,random_state = sss)
            input_train = train_x_using_iso.iloc[tri]
            input_val = train_x_using_iso.iloc[vai]

            true_train = what_is_train_y[tri]
            true_val = what_is_train_y[vai]

            globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].fit(input_train, true_train)
            output_train = globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_train)
            output_val = globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_val)

            f1_train = f1_score(true_train, output_train, average='macro')
            f1_val = f1_score(true_val, output_val, average='macro')

            # 모델 예측
            mean_val_f1 += f1_val
            print('F1 score of train and val', f1_train, f1_val)

            kkk_train = list(globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(the_real_train_x))
            kkk_test = list(globals()['knc_train_{}'.format((1 + rrr%nn_ssplit))].predict(what_is_test_x))

            hundred_df_train.iloc[:,(rrr+1)] = kkk_train
            hundred_df_test.iloc[:,(rrr+1)] = kkk_test
            
            rrr += 1
        print(' ')

        mean_f1_score = mean_val_f1/nn_ssplit
        print('이상치 제거한 것의 평균 f1 score: ', mean_f1_score)
        print('온전한 Train data의 Confusion matrix')
        print(confusion_matrix(the_real_train_y, kkk_train))
        
        print(' ')
    return hundred_df_train, hundred_df_test

In [ ]:
clf2 = IsolationForest(max_samples=100, contamination = 0.005, random_state=42)
clf2.fit(except_a_train_x.iloc[:,range(4,19)].values)

except_a_iso_train_y_value = clf2.predict(except_a_train_x.iloc[:,range(4,19)].values)
except_a_train_x_iso = except_a_train_x.loc[except_a_iso_train_y_value == 1]
except_a_train_y_getting_c_iso = except_a_train_y_getting_c[except_a_iso_train_y_value == 1]

workit_NuSVC(except_a_train_x_iso, except_a_train_x, except_a_train_y_getting_c, except_a_train_y_getting_c_iso, except_a_test_x, how_many_sssss, nn_ssplitss, 0.42, 'rbf', 'scale')

0 1 번째 시작!
F1 score of train and val 0.9722323049001815 0.9740863787375416
0 2 번째 시작!
F1 score of train and val 0.9791031185977691 0.9484126984126984
0 3 번째 시작!
F1 score of train and val 0.9661612832344539 1.0
0 4 번째 시작!
F1 score of train and val 0.9731379731379731 0.9105882352941176
0 5 번째 시작!
F1 score of train and val 0.9865689865689866 0.9415384615384615
 
이상치 제거한 것의 평균 f1 score:  0.9549251547965639
온전한 Train data의 Confusion matrix
[[114   0]
 [  4  75]]
 


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   1   1   1   0   1
 4      0.0   0   0   0   0   0
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   0
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   0   0   0   0
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   1
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   0
 120    0.0   1   1   1   1   1
 121    0.0   1   1   1   1   1
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
workit_NuSVC(bc_iso_train_x, except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, except_a_test_x, how_many_sssss, nn_ssplitss, 0.42, 'rbf', 'scale')

0 1 번째 시작!
F1 score of train and val 0.9792811839323468 0.9684569479965899
0 2 번째 시작!
F1 score of train and val 0.9650549137070319 0.9425465838509317
0 3 번째 시작!
F1 score of train and val 0.9505882352941177 0.9708431836091411
0 4 번째 시작!
F1 score of train and val 0.9709486166007906 0.9455882352941176
0 5 번째 시작!
F1 score of train and val 0.9789203817120068 0.9150275373721479
 
이상치 제거한 것의 평균 f1 score:  0.9484924976245857
온전한 Train data의 Confusion matrix
[[113   1]
 [  6  73]]
 


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   1   0   1   1   0
 4      0.0   0   0   0   0   0
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   0
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   0   0   0   0
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   1   0   1   1
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   0
 120    0.0   1   1   1   1   1
 121    0.0   1   1   1   1   1
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
clf_checking_b = IsolationForest(max_samples=50, contamination = 0.04, random_state=42)
clf_checking_c = IsolationForest(max_samples=50, contamination = 0.05, random_state=42)
except_a_train_x_only_b = except_a_train_x.iloc[except_a_train_y_getting_c == 0,range(4,19)]
except_a_train_x_only_c = except_a_train_x.iloc[except_a_train_y_getting_c == 1,range(4,19)]
who_is_outier_b = clf_checking_b.fit_predict(except_a_train_x_only_b.values)
who_is_outier_c = clf_checking_c.fit_predict(except_a_train_x_only_c.values)
except_a_train_x.iloc[:,0] = except_a_train_y_getting_c
bc_iso_train_x = except_a_train_x.drop(index = except_a_train_x_only_b.iloc[who_is_outier_b == (-1)].index) 
bc_iso_train_x = bc_iso_train_x.drop(index = except_a_train_x_only_c.iloc[who_is_outier_c == (-1)].index)
bc_iso_train_y = np.array(bc_iso_train_x.iloc[:,0])
bc_iso_train_x.iloc[:,0] = np.zeros(len(bc_iso_train_x))
bc_iso_train_x = bc_iso_train_x.astype(int)
except_a_train_x.iloc[:,0]  = np.zeros(len(except_a_train_x))
except_a_train_x = except_a_train_x.astype(int)

doing_onehot = pd.concat([bc_iso_train_x , except_a_train_x], axis=0)
doing_onehot = pd.concat([doing_onehot , except_a_test_x], axis=0)

doing_onehot = pd.concat([bc_iso_train_x , except_a_train_x], axis=0)
doing_onehot = pd.concat([doing_onehot , except_a_test_x], axis=0)
doing_onehot= pd.get_dummies(doing_onehot.astype(str))
onehot_bc_iso_train_x = doing_onehot.iloc[range(0,len(bc_iso_train_x))]
onehot_except_a_train_x = doing_onehot.iloc[range(len(bc_iso_train_x), (len(bc_iso_train_x) + len(except_a_train_x)))]
onehot_except_a_test_x = doing_onehot.iloc[range(len(bc_iso_train_x) + len(except_a_train_x), (len(bc_iso_train_x)  + len(except_a_train_x) + len(except_a_test_x)))]

workit_NuSVC(onehot_bc_iso_train_x, onehot_except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, onehot_except_a_test_x, how_many_sssss, nn_ssplitss, 0.42, 'rbf', 'scale')

0 1 번째 시작!
F1 score of train and val 0.9721062618595826 0.9353146853146852
0 2 번째 시작!
F1 score of train and val 0.9859893252001525 0.9708431836091411
0 3 번째 시작!
F1 score of train and val 0.9789327855539101 0.9407051282051282
0 4 번째 시작!
F1 score of train and val 0.9855769230769231 0.945906432748538
0 5 번째 시작!
F1 score of train and val 0.9786692289214508 0.8875
 
이상치 제거한 것의 평균 f1 score:  0.9360538859754985
온전한 Train data의 Confusion matrix
[[112   2]
 [  6  73]]
 


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   0   0   0   1   0
 4      0.0   0   0   0   0   0
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   0
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   0   0   0   0
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   0
 120    0.0   1   1   1   1   1
 121    0.0   1   1   1   1   1
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
for jb in range(100):
    b_contamination = (jb+1)/1000
    for kc in range(100):
        c_contamination = (kc+1)/1000
        print(b_contamination, c_contamination)
        clf_checking_b = IsolationForest(max_samples=50, contamination = b_contamination, random_state=42)
        clf_checking_c = IsolationForest(max_samples=50, contamination = c_contamination, random_state=42)
        except_a_train_x_only_b = except_a_train_x.iloc[except_a_train_y_getting_c == 0,range(4,19)]
        except_a_train_x_only_c = except_a_train_x.iloc[except_a_train_y_getting_c == 1,range(4,19)]
        who_is_outier_b = clf_checking_b.fit_predict(except_a_train_x_only_b.values)
        who_is_outier_c = clf_checking_c.fit_predict(except_a_train_x_only_c.values)
        except_a_train_x.iloc[:,0] = except_a_train_y_getting_c
        bc_iso_train_x = except_a_train_x.drop(index = except_a_train_x_only_b.iloc[who_is_outier_b == (-1)].index) 
        bc_iso_train_x = bc_iso_train_x.drop(index = except_a_train_x_only_c.iloc[who_is_outier_c == (-1)].index)
        bc_iso_train_y = np.array(bc_iso_train_x.iloc[:,0])
        bc_iso_train_x.iloc[:,0] = np.zeros(len(bc_iso_train_x))

        workit_NuSVC(bc_iso_train_x, except_a_train_x, bc_iso_train_y, except_a_test_x, how_many_sssss, nn_ssplitss, 0.42, 'rbf', 'scale')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 
 
0.9364616847420446
 
0.012 0.071
0 1 번째 시작!
F1 score of train and val 0.9718148924014474 0.9383333333333334
0 2 번째 시작!
F1 score of train and val 0.9786692289214508 0.9149425287356321
0 3 번째 시작!
F1 score of train and val 0.9712732919254659 0.9425465838509317
0 4 번째 시작!
F1 score of train and val 0.9714671293618662 0.9425465838509317
0 5 번째 시작!
F1 score of train and val 0.9786692289214508 0.943939393939394
 
 
0.9364616847420446
 
0.012 0.072
0 1 번째 시작!
F1 score of train and val 0.9718148924014474 0.9383333333333334
0 2 번째 시작!
F1 score of train and val 0.9786692289214508 0.9149425287356321
0 3 번째 시작!
F1 score of train and val 0.9712732919254659 0.9425465838509317
0 4 번째 시작!
F1 score of train and val 0.9714671293618662 0.9425465838509317
0 5 번째 시작!
F1 score of train and val 0.9786692289214508 0.943939393939394
 
 
0.9364616847420446
 
0.012 0.073
0 1 번째 시작!
F1 score of train and val 0.9718148924014474 0.9383333333333334
0 2 번째 시작!
F1 score of train a

KeyboardInterrupt: ignored

In [ ]:
submission_NuSVC = pd.read_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/sample_submission.csv')
submission_NuSVC.iloc[after_test_A == 0, 1] = round(workit_NuSVC(except_a_train_x_iso, except_a_train_x, except_a_train_y_getting_c_iso, except_a_test_x, how_many_sssss, nn_ssplitss, 0.42, 'rbf', 'scale')[1].iloc[:,range(1,6)].mean(axis=1))
submission_NuSVC = submission_NuSVC.replace(0,'B')
submission_NuSVC = submission_NuSVC.replace(1,'C')
submission_NuSVC.to_csv('/content/gdrive/MyDrive/testtest/유전체 정보 분류/데이터/submission_NuSVC_1.csv', index=False)

0 1 번째 시작!
F1 score of train and val 0.9654049654049653 0.9740863787375416
0 2 번째 시작!
F1 score of train and val 0.9722323049001814 0.9484126984126984
0 3 번째 시작!
F1 score of train and val 0.9595021037868163 1.0
0 4 번째 시작!
F1 score of train and val 0.9731379731379731 0.9105882352941176
0 5 번째 시작!
F1 score of train and val 0.9731379731379731 0.9415384615384615
 
 
0.9549251547965639
 


In [ ]:
from sklearn.neural_network import MLPClassifier
def workit_MLPClassifier(train_x_using_iso, the_real_train_x, the_real_train_y, what_is_train_y, what_is_test_x,  how_many_sss, nn_ssplit, the_hls=(100,), the_act = 'relu', the_solver = 'adam', the_bs='auto', lrlr='constant', the_maxi=2000):
    hundred_df_train = pd.DataFrame({'zeros':np.zeros(len(the_real_train_x))})
    hundred_df_test = pd.DataFrame({'zeros':np.zeros(len(what_is_test_x))})

    for i in range(0,(how_many_sss*nn_ssplit)):
        globals()['s{}'.format(i)] = np.zeros(len(what_is_test_x))
        ya_qustn = 's'+str(i)
        hundred_df_train = pd.concat([hundred_df_train, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)
        hundred_df_test = pd.concat([hundred_df_test, pd.DataFrame({ya_qustn: globals()['s{}'.format(i)]})], axis=1)

    antms_val = 0
    antms_f1 = 0
    for sss in range(how_many_sss):
        rrr = 0
        seed_everything(sss) # Seed 고정

        cv = KFold(n_splits=nn_ssplit, shuffle=True, random_state=sss)
        mean_val_f1 = 0

        for tri, vai in cv.split(what_is_train_y):
            print(sss, (1 + rrr%nn_ssplit),'번째 시작!')

            globals()['mlpc_train_{}'.format((1 + rrr%nn_ssplit))] = MLPClassifier(hidden_layer_sizes=the_hls, activation=the_act, solver=the_solver, batch_size=the_bs, learning_rate=lrlr, max_iter=the_maxi, random_state = sss)#
            input_train = train_x_using_iso.iloc[tri]
            input_val = train_x_using_iso.iloc[vai]

            true_train = what_is_train_y[tri]
            true_val = what_is_train_y[vai]

            globals()['mlpc_train_{}'.format((1 + rrr%nn_ssplit))].fit(input_train, true_train)
            output_train = globals()['mlpc_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_train)
            output_val = globals()['mlpc_train_{}'.format((1 + rrr%nn_ssplit))].predict(input_val)

            f1_train = f1_score(true_train, output_train, average='macro')
            f1_val = f1_score(true_val, output_val, average='macro')

            # 모델 예측
            mean_val_f1 += f1_val
            print('F1 score of train and val', f1_train, f1_val)

            kkk_train = list(globals()['mlpc_train_{}'.format((1 + rrr%nn_ssplit))].predict(the_real_train_x))
            kkk_test = list(globals()['mlpc_train_{}'.format((1 + rrr%nn_ssplit))].predict(what_is_test_x))

            hundred_df_train.iloc[:,(rrr+1)] = kkk_train
            hundred_df_test.iloc[:,(rrr+1)] = kkk_test
            
            rrr += 1
        print(' ')

        mean_f1_score = mean_val_f1/nn_ssplit
        print('이상치 제거한 것의 평균 f1 score: ', mean_f1_score)
        print('온전한 Train data의 Confusion matrix')
        print(confusion_matrix(the_real_train_y, kkk_train))
        
        print(' ')
    return hundred_df_train, hundred_df_test

In [ ]:
clf2 = IsolationForest(max_samples=100, contamination = 0.001, random_state=42)
clf2.fit(except_a_train_x.iloc[:,range(4,19)].values)

except_a_iso_train_y_value = clf2.predict(except_a_train_x.iloc[:,range(4,19)].values)
except_a_train_x_iso = except_a_train_x.loc[except_a_iso_train_y_value == 1]
except_a_train_y_getting_c_iso = except_a_train_y_getting_c[except_a_iso_train_y_value == 1]

workit_MLPClassifier(bc_iso_train_x, except_a_train_x, except_a_train_y_getting_c, bc_iso_train_y, except_a_test_x, how_many_sssss, nn_ssplitss, (200,),'logistic', 'adam', 'auto', 'adaptive', 20000)

0 1 번째 시작!
F1 score of train and val 0.993093727977449 0.9684569479965899
0 2 번째 시작!
F1 score of train and val 0.9791242603550295 0.9716475095785441
0 3 번째 시작!
F1 score of train and val 0.9791242603550295 1.0
0 4 번째 시작!
F1 score of train and val 0.9855769230769231 0.9186813186813186
0 5 번째 시작!
F1 score of train and val 0.9930106257378984 0.8885448916408669
 
이상치 제거한 것의 평균 f1 score:  0.9494661335794639
온전한 Train data의 Confusion matrix
[[110   4]
 [  4  75]]
 


(     zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   0   0   0   0   0
 3      0.0   1   1   1   1   1
 4      0.0   0   0   0   0   0
 ..     ...  ..  ..  ..  ..  ..
 188    0.0   0   0   0   0   0
 189    0.0   0   0   0   0   0
 190    0.0   0   0   0   0   0
 191    0.0   1   1   1   1   1
 192    0.0   0   0   0   0   0
 
 [193 rows x 6 columns],      zeros  s0  s1  s2  s3  s4
 0      0.0   0   0   0   0   0
 1      0.0   1   1   1   1   1
 2      0.0   1   1   1   1   1
 3      0.0   0   0   0   0   0
 4      0.0   1   1   1   1   1
 ..     ...  ..  ..  ..  ..  ..
 119    0.0   0   0   0   0   0
 120    0.0   1   1   1   1   1
 121    0.0   1   1   1   1   1
 122    0.0   0   0   0   0   0
 123    0.0   0   0   0   0   0
 
 [124 rows x 6 columns])

In [ ]:
from sklearn.cluster import DBSCAN
dbmodel = DBSCAN(eps=0.8, min_samples=5)
except_a_train_x_dbmodel = dbmodel.fit_predict(except_a_train_x.iloc[:,[10,11,17]].values)

print(np.array(except_a_train_y_getting_c))
print(np.array(except_a_train_x_dbmodel))

[0 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1
 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 1 1 0 1 0 1 1 0 0 1 1 0 0 0 1 1
 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 0 0 0 0 1
 0 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 0 0 0 0
 1 0 0 0 0 0 1 0]
[ 0  1  1  2  1  0  3  1  0  1  0  1  2  3 -1  0  0  0  1  0  1  1  0  0
  0  3  1  1  0  0  1  0  0  2  1  1  3  3  4  4  1  2  1  0  5  1  2  3
  0  3  1  5  0  1  0 -1  1  4  1  2  5  4 -1  1  3  1  0  4  1  1  0  1
 -1  0  0  0  0  0  0  2  0  1  0  0  0  1  1  2  0  2  0  1  0  1  1  3
  2  0  2  0  5  1  0  4 -1  5  0  1  0 -1  5  0  1  1  3  0  1  1  0  3
  1  1  5  1  2  1  1  4  4  1  1  1  4  5  2 -1  0  0  1  0  1 -1  0  2
  0  1  0 -1  0  3  2  0  1  4  1 -1  0  1  1  0 -1  1  0  0  1  1  2  1
  1  0  3  3  4  2  1  2  0  1  3  1  1  0  0  0  2  0  0  0  4  0  1  3
  2]


In [ ]:
from sklearn.linear_model import LogisticRegression
logisticrmodel = LogisticRegression()
logisticrmodel.fit(except_a_train_x, except_a_train_y_getting_c)
print(logisticrmodel.score(except_a_train_x, except_a_train_y_getting_c))
print(logisticrmodel.coef_)

0.9689119170984456
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.94023008e-04
  -3.02280397e-01 -1.82501577e-01 -4.25668448e-01 -8.73865643e-01
  -1.32036083e+00 -1.74407879e-01  9.79848161e-01 -1.02268452e+00
  -8.68733520e-01 -8.94918076e-01 -3.90512345e-01  2.00727567e-01
   3.36687325e-01  1.52725744e+00  4.54587717e-01]]
